## 20. Нормализация: 3NF

### 🧮 Нормализация: 3NF (Третья нормальная форма)

**Постановка проблемы:**  
Если в таблице одно поле зависит не от ключа, а от другого неключевого поля — это нарушает структуру. Например, если у студента есть город, а у города — код, но всё это в одной таблице. При изменении города — надо менять код везде. Это дублирование и потенциальные ошибки.

**Краткая теория:**  
**3NF** = 2NF + отсутствие **транзитивных зависимостей**:
- Нельзя, чтобы одни неключевые поля зависели от других
- Все неключевые поля должны зависеть **только** от ключа

Решается путём выноса зависимых данных в отдельные таблицы с внешними ключами.

**Пример:**

In [ ]:
# Демонстрация 3NF в SQLite
import sqlite3
conn = sqlite3.connect(':memory:')
cursor = conn.cursor()

# Нарушение 3NF: город и его код зависят от student_city, а не от ключа
cursor.execute('''CREATE TABLE students_bad (
    id INTEGER PRIMARY KEY,
    name TEXT,
    city TEXT,
    city_code TEXT
)''')
cursor.execute("INSERT INTO students_bad VALUES (1, 'Анна', 'Москва', 'MSK')")
cursor.execute("INSERT INTO students_bad VALUES (2, 'Боб', 'Москва', 'MSK')")
cursor.execute("INSERT INTO students_bad VALUES (3, 'Вася', 'СПб', 'LED')")
print("🚫 Нарушение 3NF:")
cursor.execute("SELECT * FROM students_bad")
for row in cursor.fetchall():
    print(row)

# Нормализация: выносим города в отдельную таблицу
cursor.execute("DROP TABLE students_bad")
cursor.execute("CREATE TABLE cities (
    id INTEGER PRIMARY KEY,
    name TEXT,
    code TEXT
)")
cursor.execute("CREATE TABLE students (
    id INTEGER PRIMARY KEY,
    name TEXT,
    city_id INTEGER,
    FOREIGN KEY (city_id) REFERENCES cities(id)
)")
# Добавим города и студентов
cursor.executemany("INSERT INTO cities (name, code) VALUES (?, ?)", [
    ("Москва", "MSK"),
    ("СПб", "LED")
])
conn.commit()
city_id_msk = cursor.execute("SELECT id FROM cities WHERE name = 'Москва'").fetchone()[0]
city_id_led = cursor.execute("SELECT id FROM cities WHERE name = 'СПб'").fetchone()[0]
cursor.executemany("INSERT INTO students (name, city_id) VALUES (?, ?)", [
    ("Анна", city_id_msk),
    ("Боб", city_id_msk),
    ("Вася", city_id_led)
])
conn.commit()

# Проверка
print("\n✅ После нормализации (3NF):")
cursor.execute('''
SELECT s.name, c.name, c.code FROM students s
JOIN cities c ON s.city_id = c.id
''')
for row in cursor.fetchall():
    print(row)